## 数据准备

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../utils/dataset/UCI_Zoo_Data_Set/zoo.data.csv', header=None)
data=data.drop([0],axis=1)    # 首列是animal_name，丢弃

data.columns=list(range(len(data.columns)))    # 将列索引更改成从0开始

## ID3
熵的计算：
$$
H(D)=-\sum\limits_{k=1}^{K}p_{k}{\log}p_{k}
$$

In [3]:
def entropy(feature):
    uni_val, cnt = np.unique(feature, return_counts=True)    # 返回独特值与计数
    # 熵的计算
    H = np.sum([(-cnt[i]/np.sum(cnt))*np.log2(cnt[i]/np.sum(cnt))
                for i in range(len(uni_val))])
    return H

# entropy(data.iloc[:,1])

信息增益计算：
$$
H(D|A)=\sum\limits_{v}^{V}\frac{|D_{v}|}{|D|}H(D_{v}) \\
G(D|A)=H(D)-H(D|A) \\
$$

In [4]:
def InfoGain(dataset, f_test_col, Y_col=-1):
    entropy_before = entropy(dataset.iloc[:, Y_col])    # 分割前的熵

    uni_val, cnt = np.unique(dataset.iloc[:, f_test_col],return_counts=True)    # 计算分割特征的独特值与计数
    entropy_cond = np.sum([(cnt[i]/np.sum(cnt))*entropy(dataset.where(dataset.iloc[:, f_test_col]
                                                                      == uni_val[i]).dropna().iloc[:, Y_col]) for i in range(len(uni_val))])
    return entropy_before-entropy_cond

# InfoGain(data,0)

ID3的基本雏形。使用递归来进行分裂，注意边界条件与每轮的特征删除。返回值是一棵以字典形式储存的分类树，叶节点为label。

In [5]:
def ID3(dataset, org_dataset, f_cols, Y_col=-1, p_node_cls=None):
    '''
    dataset: 用于分割的数据
    org_dataset: 用于计算优势类别的数据，父节点数据
    f_cols: 备选特征
    '''
    # 如果数据中的Y已经纯净了，则返回Y的取值
    if len(np.unique(dataset.iloc[:, Y_col])) <= 1:
        return np.unique(dataset.iloc[:, Y_col])[0]

    # 如果传入数据为空(对应空叶节点)，则返回原始数据中数量较多的label值
    elif len(dataset) == 0:
        uni_cls, cnt = np.unique(
            org_dataset.iloc[:, Y_col], return_counts=True)
        return uni_cls[np.argmax(cnt)]

    # 如果没有特征可用于划分，则返回父节点中数量较多的label值
    # 由于初始传入的是Index类型，所以这里不能用if not
    elif len(f_cols) == 0:
        return p_node_cls

    # 否则进行分裂
    else:
        # 得到当前节点中数量最多的label，递归时会赋给下层函数的p_node_cls
        cur_uni_cls, cnt = np.unique(
            dataset.iloc[:, Y_col], return_counts=True)
        cur_node_cls = cur_uni_cls[np.argmax(cnt)]
        del cur_uni_cls, cnt

        # 根据信息增益选出最佳分裂特征
        gains = [InfoGain(dataset, f_col, Y_col) for f_col in f_cols]
        best_f = f_cols[np.argmax(gains)]

        # 更新备选特征
        f_cols = [col for col in f_cols if col != best_f]

        # 按最佳特征的不同取值，划分数据集并递归
        tree = {best_f: {}}
        for val in np.unique(dataset.iloc[:, best_f]):    # ID3对每一个取值都划分数据集
            sub_data = dataset.where(dataset.iloc[:, best_f] == val).dropna()
            tree[best_f][val] = ID3(sub_data, dataset, f_cols, Y_col, cur_node_cls)    # 分裂特征的某一取值，对应一颗子树或叶节点

        return tree
    
# ID3(data,data,list(range(len(data.columns)-1)),-1)

In [6]:
# 生成树是一个字典形式，并且树模型需要从根节点开始判定，预测也可以通过递归实现
# 树的根节点通过tree.keys()获取，通过查询键值来得到左右子树的根节点
# 为了便于查找，将输入的测试样本也变成字典形式，特征名为key，特征值为val


def predict(query, tree, default=-1):
    '''
    query：一个测试样本，字典形式，{f:val,f:val,...}
    tree：生成树
    default：查找失败时返回的默认类别
    '''
    for feature in list(query.keys()):
        if feature in list(tree.keys()):    # 如果该特征与根节点的划分特征相同
            try:
                sub_tree = tree[feature][query[feature]]    # 根据特征的取值来获取子节点

                if isinstance(sub_tree, dict):    # 判断是否还有子树
                    return predict(query, sub_tree)    # 有则继续查找
                else:
                    return sub_tree    # 是叶节点则返回结果
            except:    # 没有查到则说明是未见过的情况，只能返回default
                return default

In [7]:
train_data=data.iloc[:80].reset_index(drop=True)
test_data=data.iloc[80:].reset_index(drop=True)

# 训练模型
tree=ID3(train_data,train_data,list(range(train_data.shape[1]-1)),-1)

# DF转dict，一个条目为一个字典，返回一个字典的列表
X_test=test_data.iloc[:,:-1].to_dict(orient = "records")
Y_test=list(test_data.iloc[:,-1])
Y_pred=list()

for item in X_test:
    Y_pred.append(predict(item,tree))
    
print('acc:{}'.format(np.sum(np.array(Y_test)==np.array(Y_pred))/len(Y_test)))

acc:0.8571428571428571
